In [1]:
import sys
sys.path.append("/home/ahow/MyGitHub/oppai_challenge/utils")

from datetime import datetime, timedelta
from pyspark.sql import functions as F

from spark_delta_transform import unnest_struct, transform_column_names
from spark_delta import merge, table_exists
from logs import print_args
from default_spark import spark

23/11/16 14:59:18 WARN Utils: Your hostname, spiriel resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface enp3s0)
23/11/16 14:59:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ahow/virtual_envs/ahow/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ahow/.ivy2/cache
The jars for the packages stored in: /home/ahow/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-085ca184-0d4b-4a5f-9cb5-6d1c87027dc7;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 110ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [2]:
class UsersETL:
    def __init__(self, spark, dt_start, dt_end, pk=None):
        self.spark = spark
        self.dt_start = dt_start  # dummy varibles to be used later in real ETL
        self.dt_end = dt_end  # dummy varibles to be used later in real ETL
        
    @staticmethod
    def _set_pk(pk):
        if pk is None:
            pk = ["id_oid"]
    
    @print_args(print_kwargs=['file_name'])
    def extract(self, file_name: str):
        df = self.spark.read.format('json').options(multiLine=True).load(file_name)
        return df
    
    def transform(self, df):
        # Unnest two levels
        df = unnest_struct(df)
        df = unnest_struct(df)
        
        df = transform_column_names(df)
        
        for c in df.columns:
            if '_date' in c:
                df = df.withColumn(c, F.to_timestamp(df[c]))
        
        return df
    
    @print_args(print_kwargs=['target_tb', 'drop'])
    def load(self, df, target_tb, drop=False):
        if drop and table_exists(target_tb, self.spark):
            print(f"Dropping table {target_tb}")
            self.spark.sql(f"DROP TABLE {target_tb}").show()

        df.write.format('delta').mode('append').saveAsTable(target_tb)

In [3]:
%run ./etl_constants.ipynb

In [4]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {DATABASE}").show()

etl = UsersETL(spark, DT_START, DT_END)
df = etl.extract(file_name=f"{BASE_PATH}/users.json")
df = etl.transform(df)
etl.load(target_tb=TARGET_RAW_TB, drop=DROP_RAW)

23/11/16 14:59:21 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/16 14:59:21 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/16 14:59:22 ERROR Schema: Failed initialising database.
Unable to open a test connection to the given database. JDBC url = jdbc:derby:;databaseName=metastore_db;create=true, username = APP. Terminating connection pool (set lazyInit to true if you expect to start your database after your app). Original Exception: ------
java.sql.SQLException: Failed to start database 'metastore_db' with class loader jdk.internal.loader.ClassLoaders$AppClassLoader@5ffd2b27, see the next exception for details.
	at org.apache.derby.impl.jdbc.SQLExceptionFactory.getSQLException(Unknown Source)
	at org.apache.derby.impl.jdbc.SQLExceptionFactory.getSQLException(Unknown Source)
	at org.apache.derby.impl.jdbc.Util.seeNextException(Unknown Source)
	at org.apache.derby.impl.jdbc.EmbedConnection.bootDatabase(Unknown Source)
	at org.a

23/11/16 14:59:24 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/16 14:59:24 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/16 14:59:24 ERROR Schema: Failed initialising database.
Unable to open a test connection to the given database. JDBC url = jdbc:derby:;databaseName=metastore_db;create=true, username = APP. Terminating connection pool (set lazyInit to true if you expect to start your database after your app). Original Exception: ------
java.sql.SQLException: Failed to start database 'metastore_db' with class loader jdk.internal.loader.ClassLoaders$AppClassLoader@5ffd2b27, see the next exception for details.
	at org.apache.derby.impl.jdbc.SQLExceptionFactory.getSQLException(Unknown Source)
	at org.apache.derby.impl.jdbc.SQLExceptionFactory.getSQLException(Unknown Source)
	at org.apache.derby.impl.jdbc.Util.seeNextException(Unknown Source)
	at org.apache.derby.impl.jdbc.EmbedConnection.bootDatabase(Unknown Source)
	at org.a

23/11/16 14:59:26 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/16 14:59:26 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/16 14:59:26 ERROR Schema: Failed initialising database.
Unable to open a test connection to the given database. JDBC url = jdbc:derby:;databaseName=metastore_db;create=true, username = APP. Terminating connection pool (set lazyInit to true if you expect to start your database after your app). Original Exception: ------
java.sql.SQLException: Failed to start database 'metastore_db' with class loader jdk.internal.loader.ClassLoaders$AppClassLoader@5ffd2b27, see the next exception for details.
	at org.apache.derby.impl.jdbc.SQLExceptionFactory.getSQLException(Unknown Source)
	at org.apache.derby.impl.jdbc.SQLExceptionFactory.getSQLException(Unknown Source)
	at org.apache.derby.impl.jdbc.Util.seeNextException(Unknown Source)
	at org.apache.derby.impl.jdbc.EmbedConnection.bootDatabase(Unknown Source)
	at org.a

23/11/16 14:59:28 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/16 14:59:28 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/16 14:59:28 ERROR Schema: Failed initialising database.
Unable to open a test connection to the given database. JDBC url = jdbc:derby:;databaseName=metastore_db;create=true, username = APP. Terminating connection pool (set lazyInit to true if you expect to start your database after your app). Original Exception: ------
java.sql.SQLException: Failed to start database 'metastore_db' with class loader jdk.internal.loader.ClassLoaders$AppClassLoader@5ffd2b27, see the next exception for details.
	at org.apache.derby.impl.jdbc.SQLExceptionFactory.getSQLException(Unknown Source)
	at org.apache.derby.impl.jdbc.SQLExceptionFactory.getSQLException(Unknown Source)
	at org.apache.derby.impl.jdbc.Util.seeNextException(Unknown Source)
	at org.apache.derby.impl.jdbc.EmbedConnection.bootDatabase(Unknown Source)
	at org.a

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/ahow/virtual_envs/ahow/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ahow/virtual_envs/ahow/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df = etl.extract(file_name=f"{BASE_PATH}/users.json")
df.toPandas()

In [ ]:
df.columns